## SNR Methods

In [ ]:
import Labber
import os
import sys
from fitTools.utilities import Watt2dBm, dBm2Watt, VNA2dBm

from twpa_tune_up_helper_functions import *


In [ ]:
def calculate_mean_SNR_from_Labber_file(labber_data_file5, cutOff = 10e3):
    """
    """
    lf = Labber.LogFile(labber_data_file)
    SA_channel_name = 'HP Spectrum Analyzer - Signal'


    signal = lf.getData(name = SA_channel_name)
    linsig = dBm2Watt(signal)

    SAxdata, SAydata = lf.getTraceXY(y_channel=SA_channel_name, entry=0) # gives last trace from SA
    
    snrs = 0
    for i in range(len(signal)):
        snrs += get_signal_stats(dBm2Watt(signal[i]), SAxdata, cutOff)[0]

    snr_mean = snrs / len(signal)
    return snr_mean

**Testing SNR Code:**

In [ ]:
calculate_mean_SNR_from_Labber_file